In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading Data

In [ ]:
train = pd.read_csv("/kaggle/input/gdsc-rec-2/train.csv")
test = pd.read_csv("/kaggle/input/gdsc-rec-2/Kaggle_test.csv")

# EDA

In [ ]:
train.iloc[:,1:].describe().transpose()[['count', 'mean', '50%', 'min', 'max', 'std']]

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.head()

Do not consider ID's values


In [ ]:
train.iloc[:,1:].hist(figsize=(20,20), xrot=45)
plt.show()

In [ ]:
train.describe()

In [ ]:
train.describe(include='object')

In [ ]:
for i in train.select_dtypes(include='object'):
    if train[i].nunique() <= 22:
        sns.countplot(y=i, data=train)
        plt.show()

In [ ]:
for i in train.select_dtypes(include='object'):
    if train[i].nunique() <=4:
        display(pd.crosstab(train['House'], train[i], normalize='index'))

In [ ]:
train

# Test Dataset EDA

In [ ]:
for i in test.select_dtypes(include='object'):
    if test[i].nunique() <= 22:
        sns.countplot(y=i, data=test)
        plt.show()

In [ ]:
test.iloc[:,1:].hist(figsize=(20,20), xrot=45)
plt.show()

In [ ]:
test.iloc[:,1:].describe().transpose()[['count', 'mean', '50%', 'min', 'max', 'std']]

# STATISTICAL DATA ANALYSIS

# OUTLIER DETECTION and DISTRIBUTION

In [ ]:
for i in train.columns[1:]:
    plt.figure(figsize=(10, 10))
    if train.dtypes[i] == "float64":
        sns.histplot(train[i], kde = False, color ='red')
        plt.show()
        sns.boxplot(train[i])
        plt.show()

# IQR For Outlier Treatment

In [ ]:
def IQR(col):
    q1 = np.percentile(col, 25)
    q3 = np.percentile(col, 75)
    iqr = q3 - q1
    ub = q3 + (1.5 * iqr)
    uo = col > ub
    lb = q1 - (1.5 * iqr)
    lo = col < ub
    col = np.where(uo, round(ub, 2), col)
    col = np.where(lo, round(lb, 2), col)
    return col

In [ ]:
for i in train.columns[1:]:
    plt.figure(figsize=(10, 10))
    if train.dtypes[i] == "float64":
        train[i] = IQR(train[i])
        sns.histplot(train[i], kde = False, color ='red')
        plt.show()
        sns.boxplot(train[i])
        plt.show()


# Correlation Matrix and Heatmap

In [ ]:
cr = pd.read_csv("/kaggle/input/gdsc-rec-2/train.csv")
for i in cr.columns:
    if cr.dtypes[i] == "float64":
        pass
    else:
        cr = cr.drop([i], axis=1)
co = cr.corr()
plt.figure(figsize=(15,14))
sns.heatmap(co,annot=True)
plt.title('Correlation Matrix')
plt.show()
co

In [ ]:
for i in train.columns:
    if train.dtypes[i] == "object":
        print(f"Unique values of {i}")
        print(train[i].unique(),"\n")

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
for i in train.columns:
    if train.dtypes[i] == "object":
        train[i] = le.fit_transform(train[i])

# Splitting Training set and test set

In [ ]:
from sklearn.model_selection import train_test_split
feature = train.iloc[:, 1:-1]
target = train.iloc[:, -1]
xtrain, xtest, ytrain, ytest = train_test_split(
    feature, target, test_size=0.2, random_state=0)


# LAZY PREDICT

In [ ]:
pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True,custom_metric=None)
models, predictions = clf.fit(xtrain, xtest,ytrain, ytest)
print(models)

In [ ]:
ytest

In [ ]:
tr = pd.read_csv("/kaggle/input/gdsc-rec-2/Kaggle_test.csv")
for i in tr.columns:
    if tr.dtypes[i] == "float64":
        pass
    else:
        tr = tr.drop([i], axis=1)
to = cr.corr()
plt.figure(figsize=(15,14))
sns.heatmap(to,annot=True)
plt.title('Correlation Matrix')
plt.show()

Correlation Matrix

In [ ]:
to 

In [ ]:
for i in test.columns[1:]:
    plt.figure(figsize=(10, 10))
    if test.dtypes[i] == "float64":
        sns.histplot(test[i], kde = False, color ='red')
        plt.show()
        sns.boxplot(test[i])
        plt.show()

# Test Data EDA & OUTLIER TREATMENT

In [ ]:
for i in test.columns[1:]:
    plt.figure(figsize=(10, 10))
    if test.dtypes[i] == "float64":
        test[i] = IQR(test[i])
        sns.histplot(test[i], kde = False, color ='red')
        plt.show()
        sns.boxplot(test[i])
        plt.show()

# LABEL ENCODING FOR TEST DATASET

In [ ]:
ke = LabelEncoder()
for i in test.columns:
    if test.dtypes[i] == "object":
        test[i] = ke.fit_transform(test[i])
tf = test.iloc[:, 1:]

# Scaling

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(xtrain)
X_test = scaler.transform(xtest)

# MODELS and Metrics

**Selected Model: 1.Perceptron 2. NuSVC**

# Extra Trees Classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators = 100,criterion ='entropy', max_features = 14)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred1 = model.predict(tf)
pred11 = le.inverse_transform(pred1)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred11})
print(submission)
'''submission.to_csv('submission_ETC.csv', index=False)'''

# Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='entropy',max_depth=2)
model.fit(xtrain, ytrain)

YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))


pred2 = model.predict(tf)
pred22 = le.inverse_transform(pred2)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred22})
print(submission)
'''submission.to_csv('submission.csv', index=False)'''


# Perceptron Model (Selected)

In [ ]:
from sklearn.linear_model import Perceptron
model = Perceptron()
model.fit(xtrain, ytrain)

YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))


pred3 = model.predict(tf)
pred33 = le.inverse_transform(pred3)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred33})
print(submission)
submission.to_csv('submission_PRC.csv', index=False)

# NuSV Classifier (Selected)

In [ ]:
from sklearn.svm import NuSVC
model = NuSVC(gamma='scale')
model.fit(xtrain,ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred5 = model.predict(tf)
pred55 = le.inverse_transform(pred5)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred55})
print(submission)
submission.to_csv('submission_NuSV.csv', index=False)

# XGB Classifier

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(xtrain,ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred6 = model.predict(tf)
pred66 = le.inverse_transform(pred6)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred66})
print(submission)
'''submission.to_csv('submissionXGBC.csv', index=False)'''

# Passive Aggressive Classifier

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
model = PassiveAggressiveClassifier(C = 0.5, random_state = 5)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred6 = model.predict(tf)
pred66 = le.inverse_transform(pred6)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred66})
print(submission)
'''submission.to_csv('submission_PAC.csv', index=False)'''

# Ada Boost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model = abc.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred8 = model.predict(tf)
pred88 = le.inverse_transform(pred8)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred88})
print(submission)
'''submission.to_csv('submission_ABC.csv', index=False)'''

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100)  
clf.fit(xtrain, ytrain)
YP = clf.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred9 = clf.predict(tf)
pred99 = le.inverse_transform(pred9)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred99})
print(submission)
'''submission.to_csv('submission_RFC.csv', index=False)'''

# KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, ytrain)
YP = knn.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = knn.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submission_knn.csv', index=False)'''

# GaussianNB Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xtrain, ytrain)
YP = gnb.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = gnb.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissiongn.csv', index=False)'''

# Logistic Regression Model 

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial',solver ='newton-cg')
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submission_LogR.csv', index=False)'''

# Ridge Classifier

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
model = RidgeClassifierCV().fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submission_RidgeC.csv', index=False)'''

# Linear SVC Model

In [ ]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(xtrain, ytrain)
YP = lsvc.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = lsvc.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submission_LSVC.csv', index=False)'''

# SVC Model

In [ ]:
from sklearn.svm import SVC
X_train = scaler.fit_transform(xtrain)
X_test = scaler.transform(xtest)
svm = SVC(kernel="rbf", gamma=0.5, C=1.0)
svm.fit(X_train, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = svm.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submission_SVC.csv', index=False)'''

# Bernoulli Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB(force_alpha=True)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionBNB.csv', index=False)'''

# Nearest Centroid Model

In [ ]:
from sklearn.neighbors import NearestCentroid
model = NearestCentroid()
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionNC.csv', index=False)'''

# BAGGING CLASSIFIERS

WITH PERCEPTRON MODEL

In [ ]:
from sklearn.ensemble import BaggingClassifier
base_classifier = Perceptron()
model = BaggingClassifier(base_estimator=base_classifier, n_estimators=150)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionBC_PR_150.csv', index=False)'''


WITH NUSVC MODEL

In [ ]:
base_classifier = NuSVC(gamma='scale')
model = BaggingClassifier(base_estimator=base_classifier, n_estimators=100)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionBC_NS.csv', index=False)'''

WITH ADABOOST CLASSIFIER

In [ ]:
base_classifier = AdaBoostClassifier(n_estimators=100,learning_rate=1)
model = BaggingClassifier(base_estimator=base_classifier, n_estimators=100)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionBC_ABC_100.csv', index=False)'''

WITH XGBC CLASSIFIER

In [ ]:
base_classifier = xgb.XGBClassifier()
model = BaggingClassifier(base_estimator=base_classifier, n_estimators=100)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionBC_XGBC.csv', index=False)'''

Base Classifier with Random Forest Classifier

In [ ]:
base_classifier= RandomForestClassifier(n_estimators = 100)  
model = BaggingClassifier(base_estimator=base_classifier, n_estimators=100)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionBC_RFC.csv', index=False)'''

With Extra Trees Classifier

In [ ]:
base_classifier= ExtraTreesClassifier(n_estimators = 100,criterion ='entropy', max_features = 8)
model = BaggingClassifier(base_estimator=base_classifier, n_estimators=100)
model.fit(xtrain, ytrain)
YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)
'''submission.to_csv('submissionBC_ETC.csv', index=False)'''

# LGB Classifier

In [ ]:
#pip install lightgbm

In [ ]:
'''import lightgbm as lgb
from lightgbm import LGBMClassifier

xtrain.rename(columns = {'What are you curious about?':'wayca'}, inplace = True)
xtrain.rename(columns = {'If you were a principal, how would you choose your students?':'que'}, inplace = True)
#X_test.rename(columns = {'What are you curious about?':'wayca'}, inplace = True)
#X_test.rename(columns = {'If you were a principal, how would you choose your students?':'que'}, inplace = True)
model = LGBMClassifier()
model.fit(xtrain, ytrain)

YP = model.predict(X_test)
accuracy = accuracy_score(ytest, YP)
print("Accuracy:", accuracy)
print("Confusion Matrix")
print(confusion_matrix(ytest, YP))
print("CLASSIFICATION REPORT")
print(classification_report(ytest, YP,target_names = ["Gryffindor","Hufflepuff","Ravenclaw","Slytherine"]))


tf.rename(columns = {'What are you curious about?':'wayca'}, inplace = True)
tf.rename(columns = {'If you were a principal, how would you choose your students?':'que'}, inplace = True)
pred = model.predict(tf)
pred = le.inverse_transform(pred)
submission = pd.DataFrame({'ID': test.iloc[:,0], 'house': pred})
print(submission)'''
'''submission.to_csv('submission.csv', index=False)'''